<a href="https://colab.research.google.com/github/safdarjung/projects/blob/main/content_based_movie_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
movies = pd.read_csv('/content/movies_metadata.csv')
credits = pd.read_csv('/content/credits.csv')
keywords = pd.read_csv('/content/keywords.csv')

ParserError: ignored

In [ ]:
movies['id'] = movies['id'].astype(str)
credits['id'] = credits['id'].astype(str)
keywords['id'] = keywords['id'].astype(str)
movies['genres'] = movies['genres'].astype(str)
movies =movies.merge(credits,on='id')
movies = movies.merge(keywords,on='id')

NameError: ignored

In [ ]:
movies = movies[['id','title','genres','overview','keywords','release_date','cast','crew','popularity']]

In [ ]:
movies.dropna(inplace=True)
movies.drop_duplicates(inplace=True)

In [ ]:
import ast
import json
def find_genres(obj):
    L =[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

movies['genres']=movies['genres'].apply(find_genres)
movies['keywords']=movies['keywords'].apply(find_genres)
def find_genres2(obj):
    L =[]
    c=0
    for i in ast.literal_eval(obj):
        if c!=3:
            L.append(i['name'])
            c+=1
        else:
            break
    return L
movies['cast']=movies['cast'].apply(find_genres2)

In [ ]:
def find_director(obj):
    L =[]
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L
movies['crew']=movies['crew'].apply(find_director)

In [ ]:
movies['overview']=movies['overview'].apply(lambda x: x.split())
movies['genres'] = movies['genres'].apply(lambda x: [i.replace(" ","") for i in x])
movies['overview'] = movies['overview'].apply(lambda x: [i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x: [i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x: [i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x: [i.replace(" ","") for i in x])

In [ ]:
movies['tags'] = movies['genres']+movies['overview']+movies['keywords']+movies['cast']+movies['crew']+movies['popularity'].apply(lambda x: [str(x)])

In [ ]:
df = movies[['id','title','tags']]
df['tags']=df['tags'].apply(lambda x:" ".join(x))
df['tags']=df['tags'].apply(lambda x:x.lower())

C:\Users\safda\AppData\Local\Temp\ipykernel_21924\3914501616.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags']=df['tags'].apply(lambda x:" ".join(x))
C:\Users\safda\AppData\Local\Temp\ipykernel_21924\3914501616.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags']=df['tags'].apply(lambda x:x.lower())



stemmer converts similar words to a single word: lover,loving,love,loved --> love

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
    y =[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)
df['tags']=df['tags'].apply(stem)

C:\Users\safda\AppData\Local\Temp\ipykernel_21924\3719061741.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags']=df['tags'].apply(stem)


Text vectorization creates vectors for each movie which helps us to calculate the similar movies
vectors are like coordinates where eaxh axis can be thought of as a tag.
we will now find 5000 most common tags (action,animation,comedy,salmankhan,games,etc) and find the frequency of each of these tags in every movie.this is how every movie is converted to a unique vector.
Similar movies will be the vectors with the least distance from the main movie vector.
for the 5000 most common tags we do not consider "stop words" like a,is,am,are,from,for,this,that.

for vectorization we use sklearn class CountVectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vectors=cv.fit_transform(df['tags']).toarray()

In [ ]:
# len(cv.get_feature_names())

Now we will calculate the cosine distance (anglular dist) for each movie
dist proportional 1/similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity=cosine_similarity(vectors)
similarity

In [ ]:
def recommend(movie):
    movie_idx=df[df['title']==movie].index[0]
    distances=similarity[movie_idx]
    similar_movies=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    for i in similar_movies:
        print(df.iloc[i[0:5]].title)
recommend('Batman')

In [ ]:
df[df['title']=='Batman']

In [ ]:
import pickle
pickle.dump(df,open('movies.pkl','wb'))

In [ ]:
sorted(similarity[0],reverse=True)

In [ ]:
df.head()